### 1. Weather model

For this assignment, you'll revisit the historical temperature dataset. To complete this assignment, submit a link a Jupyter notebook containing your solutions to the following tasks:

* First, load the dataset from the **weatherinszeged** table from Thinkful's database.
* Like in the previous checkpoint, build a linear regression model where your target variable is the difference between the *apparenttemperature* and the *temperature*. As explanatory variables, use *humidity* and *windspeed*. Now, estimate your model using OLS. What are the R-squared and adjusted R-squared values? Do you think they are satisfactory? Why? 
* Next, include the interaction of *humidity* and *windspeed* to the model above and estimate the model using OLS. Now, what is the R-squared of this model? Does this model improve upon the previous one? 
* Add *visibility* as an additional explanatory variable to the first model and estimate it. Did R-squared increase? What about adjusted R-squared? Compare the differences put on the table by the interaction term and the *visibility* in terms of the improvement in the adjusted R-squared. Which one is more useful?
* Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = "{:.3f}".format

%load_ext nb_black

import warnings

warnings.filterwarnings(action="ignore")

<IPython.core.display.Javascript object>

In [2]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "weatherinszeged"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
szeged_df = pd.read_sql_query("select * from weatherinszeged", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


szeged_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


<IPython.core.display.Javascript object>

In [3]:
szeged_df["target_temp"] = szeged_df["apparenttemperature"] - szeged_df["temperature"]

<IPython.core.display.Javascript object>

In [4]:
y = szeged_df["target_temp"]
X = szeged_df[["humidity", "windspeed"]]

<IPython.core.display.Javascript object>

In [5]:
X = sm.add_constant(X)

results = sm.OLS(y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            target_temp   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Tue, 07 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:21:40   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

Our R-squared and adj-R-squared values are low at just ~0.29.  This probably means that we need more features to explain the variation in the data.

In [6]:
szeged_df["wind_hum_interaction"] = szeged_df["windspeed"] * szeged_df["humidity"]

<IPython.core.display.Javascript object>

In [7]:
y = szeged_df["target_temp"]
X2 = szeged_df[["humidity", "windspeed", "wind_hum_interaction"]]

<IPython.core.display.Javascript object>

In [8]:
X2 = sm.add_constant(X2)

results = sm.OLS(y, X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            target_temp   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Tue, 07 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:23:56   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    0.0839      0.033      2.511      0.012       0.018       0.149
humidity                 0.1775      0.043      4.133      0.000       0.093       0.262
windspeed                0.0905      0.002     36.797      0.000       0.086       0.095
wind_hum_interaction    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

The R-squared terms are higher now at about 0.34.  This suggests adding the interaction term was a positive step toward explaining more variance, but it is not a good model yet.

In [9]:
y = szeged_df["target_temp"]
X3 = szeged_df[["humidity", "windspeed", "visibility"]]

<IPython.core.display.Javascript object>

In [10]:
X3 = sm.add_constant(X3)

results = sm.OLS(y, X3).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            target_temp   R-squared:                       0.304
Model:                            OLS   Adj. R-squared:                  0.303
Method:                 Least Squares   F-statistic:                 1.401e+04
Date:                Tue, 07 Apr 2020   Prob (F-statistic):               0.00
Time:                        23:28:05   Log-Likelihood:            -1.6938e+05
No. Observations:               96453   AIC:                         3.388e+05
Df Residuals:                   96449   BIC:                         3.388e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.5756      0.028     56.605      0.000       1.521       1.630
humidity      -2.6066      0.025   -102.784      0.000      -2.656      -2.557
windspeed     -0.1199      0.001   -179.014      0.000      -0.121      -0.119
visibility     0.0540      0.001     46.614      0.000       0.052       0.056
==============================================================================
Omnibus:                     3833.895   Durbin-Watson:                   0.282
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4584.022
Skew:                          -0.459   Prob(JB):                         0.00
Kurtosis:                       3.545   Cond. No.                         131.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<IPython.core.display.Javascript object>

It looks like all of the coefficients are statistically significant, and the R-squared and adj-R-squared went up to 0.3, but this is not as good of a model as the one with an interaction term.

The model with the best BIC and AIC scores is the second one with the interaction term.  This is also the model with the best R-squared values.  

###  2. House prices model

In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

* Load the **houseprices** data from Thinkful's database.
* Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.
* Do you think your model is satisfactory? If so, why?
* In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables. 
* For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

import statsmodels.stats.api as sms
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%load_ext nb_black

import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [3]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
houseprices_df = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


houseprices_df.head(5)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [5]:
houseprices_df["bsmtqual"].unique()

array(['Gd', 'TA', 'Ex', None, 'Fa'], dtype=object)

<IPython.core.display.Javascript object>

In [4]:
houseprices_df["log_price"] = np.log(houseprices_df["saleprice"])

houseprices_df = houseprices_df[
    [
        "log_price",
        "overallqual",
        "grlivarea",
        "garagearea",
        "fullbath",
        "yearbuilt",
        "exterqual",
        "garagefinish",
        "centralair",
    ]
]

hp_df = houseprices_df.dropna()

drop_ids = hp_df.sort_values("grlivarea", ascending=False).iloc[:4].index
print(f"Dropping house ids {list(drop_ids)}")
hp_df = hp_df.drop(index=drop_ids)

hp_df["centralair"] = (hp_df["centralair"] == "Y").astype(int)

quality_map = {"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
hp_df[["exterqual"]] = hp_df[["exterqual"]].replace(quality_map)

garage_df = pd.get_dummies(hp_df["garagefinish"], drop_first=True)

hp_df = pd.concat([hp_df, garage_df], axis=1)
hp_df.drop(columns=["garagefinish"], inplace=True)

Dropping house ids [1298, 523, 1182, 691]


<IPython.core.display.Javascript object>

In [5]:
X = hp_df.drop(columns=["log_price", "RFn"])
y = hp_df["log_price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=32
)

<IPython.core.display.Javascript object>

In [6]:
X_train_const = sm.add_constant(X_train)
lm = sm.OLS(y_train, X_train_const).fit()

lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.834
Method:                 Least Squares   F-statistic:                     692.4
Date:                Thu, 09 Apr 2020   Prob (F-statistic):               0.00
Time:                        00:22:15   Log-Likelihood:                 501.86
No. Observations:                1100   AIC:                            -985.7
Df Residuals:                    1091   BIC:                            -940.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.0321      0.492     14.297      0.000       6.067       7.997
overallqual     0.0901      0.006     15.229      0.000       0.079       0.102
grlivarea       0.0003   1.47e-05     21.794      0.000       0.000       0.000
garagearea      0.0003   3.25e-05      8.230      0.000       0.000       0.000
fullbath       -0.0422      0.013     -3.275      0.001      -0.068      -0.017
yearbuilt       0.0018      0.000      7.051      0.000       0.001       0.002
exterqual       0.0716      0.012      5.770      0.000       0.047       0.096
centralair      0.1788      0.023      7.746      0.000       0.134       0.224
Unf            -0.0586      0.012     -4.737      0.000      -0.083      -0.034
==============================================================================
Omnibus:                      112.703   Durbin-Watson:                   2.039
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              311.139
Skew:                          -0.537   Prob(JB):                     2.74e-68
Kurtosis:                       5.374   Cond. No.                     2.71e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.71e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

**Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.**

The F-test p-value means we reject the null hypothesis that this model is indistinguishable from a reduced model at the p<0.01 level.  This means that the features are working to explain some of the variance in the target.

The R-squared and adjusted R-squared values indicate that the model is accounting for about 83.4% of the variance.  We conclude that there may still be room for improvement.  We can be reasonably confident that we aren't overfitting.

The AIC and BIC values are smaller, aka more negative, than the first houseprices model I ran that contained more features which weren't contributing significantly to the model.  Smaller AIC and BIC values are a sign that the model's performance is improving, and that cutting those features out was a positive step.



**Do you think your model is satisfactory? If so, why?**

No, there is clearly some unexplained variance left to capture, and I haven't exhausted my options.  I'm also concerned that it relies on features like overallqual and exterqual, which seem to be themselves models.  How are overallqual and exterqual determined?  Using these features in my model might make interpretation of the model less useful depending on the use case.  

**In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.**

I'll try adding bsmtqual and kitchenqual, and see if those truly are redundant to overallqual.

In [7]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "houseprices"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)
houseprices2_df = pd.read_sql_query("select * from houseprices", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


houseprices2_df.head(5)

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.0,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


<IPython.core.display.Javascript object>

In [8]:
houseprices2_df["log_price"] = np.log(houseprices2_df["saleprice"])

houseprices2_df = houseprices2_df[
    [
        "log_price",
        "overallqual",
        "bsmtqual",
        "kitchenqual",
        "grlivarea",
        "garagearea",
        "fullbath",
        "yearbuilt",
        "exterqual",
        "garagefinish",
        "centralair",
    ]
]

hp2_df = houseprices2_df.dropna()

drop_ids = hp2_df.sort_values("grlivarea", ascending=False).iloc[:4].index
print(f"Dropping house ids {list(drop_ids)}")
hp2_df = hp2_df.drop(index=drop_ids)

hp2_df["centralair"] = (hp2_df["centralair"] == "Y").astype(int)

quality_map = {"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
hp2_df[["exterqual"]] = hp2_df[["exterqual"]].replace(quality_map)
hp2_df[["kitchenqual"]] = hp2_df[["kitchenqual"]].replace(quality_map)

quality_map2 = {None: 0,"Fa": 1, "TA": 2, "Gd": 3, "Ex": 4}
hp2_df[["bsmtqual"]] = hp2_df[["bsmtqual"]].replace(quality_map2)

garage2_df = pd.get_dummies(hp2_df["garagefinish"], drop_first=True)

hp2_df = pd.concat([hp2_df, garage2_df], axis=1)
hp2_df.drop(columns=["garagefinish"], inplace=True)

Dropping house ids [1298, 523, 1182, 691]


<IPython.core.display.Javascript object>

In [9]:
X2 = hp2_df.drop(columns=["log_price", "RFn"])
y2 = hp2_df["log_price"]

X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=32
)

<IPython.core.display.Javascript object>

In [10]:
X2_train_const = sm.add_constant(X2_train)
lm2 = sm.OLS(y2_train, X2_train_const).fit()

lm2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              log_price   R-squared:                       0.838
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                     551.1
Date:                Thu, 09 Apr 2020   Prob (F-statistic):               0.00
Time:                        17:14:26   Log-Likelihood:                 528.91
No. Observations:                1076   AIC:                            -1036.
Df Residuals:                    1065   BIC:                            -981.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           7.7577      0.539     14.383      0.000       6.699       8.816
overallqual     0.0780      0.006     12.706      0.000       0.066       0.090
bsmtqual        0.0436      0.011      3.913      0.000       0.022       0.066
kitchenqual     0.0571      0.010      5.453      0.000       0.037       0.078
grlivarea       0.0003   1.43e-05     22.173      0.000       0.000       0.000
garagearea      0.0002   3.11e-05      7.815      0.000       0.000       0.000
fullbath       -0.0407      0.012     -3.288      0.001      -0.065      -0.016
yearbuilt       0.0014      0.000      4.973      0.000       0.001       0.002
exterqual       0.0282      0.013      2.176      0.030       0.003       0.054
centralair      0.1760      0.024      7.462      0.000       0.130       0.222
Unf            -0.0421      0.012     -3.532      0.000      -0.066      -0.019
==============================================================================
Omnibus:                      127.193   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              403.535
Skew:                          -0.575   Prob(JB):                     2.36e-88
Kurtosis:                       5.771   Cond. No.                     3.04e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.04e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

<IPython.core.display.Javascript object>

**For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?**

The second model in which I included bsmtqual and kitchenqual is slightly better than wihtout. The adjusted R-squared value went up by 0.3%, the AIC and BIC values went down even further, and the F-statistic increased.  